### Context Managers - Additional Uses

In [23]:
import decimal

In [24]:
decimal.getcontext().prec = 14
decimal.getcontext()

Context(prec=14, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[Inexact, Rounded], traps=[InvalidOperation, DivisionByZero, Overflow])

In [25]:
decimal.getcontext().prec = 28
decimal.getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[Inexact, Rounded], traps=[InvalidOperation, DivisionByZero, Overflow])

#### Temporarily change the precision of the decimals

In [26]:
old_prec = decimal.getcontext().prec

decimal.getcontext().prec = 4

print(decimal.Decimal(1) / decimal.Decimal(3))

decimal.getcontext().prec = old_prec

print(decimal.Decimal(1) / decimal.Decimal(3))


0.3333
0.3333333333333333333333333333


#### Implementing own context manager for changing the precision of decimals

```
Change Decimal Precisions
```

In [27]:
class Precision:
    def __init__(self, prec):
        self.prec = prec
        self.curr_prec = decimal.getcontext().prec
    
    def __enter__(self):
        # Set to the value of the instance attribute
        decimal.getcontext().prec = self.prec
        
    def __exit__(self, exc_type, exc_value, exc_trace):
        decimal.getcontext().prec = self.curr_prec
        return False


In [28]:
# Within the context manager
with Precision(3):
    print(decimal.Decimal(1) / decimal.Decimal(3))

# Outside the context manager
print(decimal.Decimal(1) / decimal.Decimal(3))

0.333
0.3333333333333333333333333333


#### The built-in context manager of the decimal class

```
Change and Reset Pattern
```

In [29]:
# Within the context manager
with decimal.localcontext() as ctx:
    ctx.prec = 3
    print(decimal.Decimal(1) / decimal.Decimal(3))

# Outside the context manager
print(decimal.Decimal(1) / decimal.Decimal(3))

0.333
0.3333333333333333333333333333


In [30]:
from time import perf_counter, sleep

#### Timer Context Manager

```
Start and Stop Pattern
```

In [31]:
class Timer:
    def __init__(self):
        self.elapsed = 0
        
    def __enter__(self):
        self.start = perf_counter()
        return self
    
    def __exit__(self, exc_type, exc_value, exc):
        self.stop = perf_counter()
        self.elapsed = self.stop - self.start
        return False
    

In [35]:
# Using aliases with custom context managers
with Timer() as timer:
    sleep(1)
    
print(timer.elapsed)

1.000456100000065


#### Redirecting Standard Out

```
Open and Close Pattern
```

In [38]:
import sys

class OutToFile:
    def __init__(self, file_name):
        self._fname = file_name
        self._curr_stdout = sys.stdout
        
    def __enter__(self):
        self._file = open(self._fname, 'w')
        sys.stdout = self._file
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        sys.stdout = self._curr_stdout
        self._file.close()
        return False
        

In [39]:
with OutToFile('test.txt'):
    print('Line 1')
    print('Line 2')


In [40]:
sys.stdout

In [41]:
sys.stdin

<_io.TextIOWrapper name='<stdin>' mode='r' encoding='utf-8'>

In [42]:
with open('test.txt') as f:
    print(f.readlines())

['Line 1\n', 'Line 2\n']


#### Creating open and close tags for printing

```
Open and Close Pattern
```

In [64]:
my_message = """Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!"""

In [65]:
class Tag:
    def __init__(self, tag):
        self._tag = tag
    
    def __enter__(self):
        print(f'<{self._tag}>', end='')
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        print(f'</{self._tag}>', end='')
        return False

In [66]:
with Tag('p'):
    print('Mastering ', end='')
    with Tag('b'):
        print('Python', end='')
    print(' Skills', end='')
    

<p>Mastering <b>Python</b> Skills</p>

In [92]:
lines = my_message.split('\n')

for line in lines:
    print()
    with Tag('p'):
        print(line, end='')



<p>Beautiful is better than ugly.</p>
<p>Explicit is better than implicit.</p>
<p>Simple is better than complex.</p>
<p>Complex is better than complicated.</p>
<p>Flat is better than nested.</p>
<p>Sparse is better than dense.</p>
<p>Readability counts.</p>
<p>Special cases aren't special enough to break the rules.</p>
<p>Although practicality beats purity.</p>
<p>Errors should never pass silently.</p>
<p>Unless explicitly silenced.</p>
<p>In the face of ambiguity, refuse the temptation to guess.</p>
<p>There should be one-- and preferably only one --obvious way to do it.</p>
<p>Although that way may not be obvious at first unless you're Dutch.</p>
<p>Now is better than never.</p>
<p>Although never is often better than *right* now.</p>
<p>If the implementation is hard to explain, it's a bad idea.</p>
<p>If the implementation is easy to explain, it may be a good idea.</p>
<p>Namespaces are one honking great idea -- let's do more of those!</p>

#### Creating a list maker class

```
Open and Close Pattern
```

#### Output:

```
Title
- Items 1
    - sub item 1a
    - sub item 1b
- Items 2
    - sub item 2a
    - sub item 2b
```

In [93]:
class ListMaker:
    def __init__(self, title, prefix='- ', indent=4):
        self._title = title
        self._prefix = prefix
        self._indent = indent
        self._current_indent = 0
        print(title)
    
    def __enter__(self):
        self._current_indent += self._indent
        return self
        
    def __exit__(self, exc_type, exc_value, exc_tb):
        self._current_indent -= self._indent
        return False
    
    def print_item(self, arg):
        s = ' ' * self._current_indent + self._prefix + str(arg)
        
        print(s)
    

In [94]:
with ListMaker('Items') as lm:
    lm.print_item('Item 1')
    lm.print_item('Item 2')
    lm.print_item('Item 3')

Items
    - Item 1
    - Item 2
    - Item 3


In [95]:
with ListMaker('Items') as lm:
    lm.print_item('Item 1')
    with lm:
        lm.print_item('Subitem 1a')
        lm.print_item('Subitem 1b')
    lm.print_item('Item 2')
    with lm:
        lm.print_item('Subitem 2a')
        with lm:
            lm.print_item('x')
            lm.print_item('y')
        lm.print_item('Subitem 2b')
    lm.print_item('Item 3')
    with lm:
        lm.print_item('Subitem 3a')
        lm.print_item('Subitem 3b')
        with lm:
            lm.print_item('x')
            lm.print_item('y')
            lm.print_item('z')
        lm.print_item('Subitem 3c')

Items
    - Item 1
        - Subitem 1a
        - Subitem 1b
    - Item 2
        - Subitem 2a
            - x
            - y
        - Subitem 2b
    - Item 3
        - Subitem 3a
        - Subitem 3b
            - x
            - y
            - z
        - Subitem 3c


#### Creating own list maker in a .txt file

In [96]:
with OutToFile('learn_map.txt'):
    with ListMaker('Technologies to learn') as lm:
        # Python
        lm.print_item('Python')
        
        with lm:
            lm.print_item('Python Fundamentals and Advance')
            with lm:
                lm.print_item('Idiomatic Python')
                lm.print_item('Iteration and Generators')
                lm.print_item('Serialization and Hashing')
                lm.print_item('Functional Programming')
                lm.print_item('Object-Oriented Programming')
        with lm:
            lm.print_item('Testing')
            with lm:
                lm.print_item('Unit Tests Fundamentals')
                lm.print_item('PyTest Framework')
        with lm:
            lm.print_item('Back-end Framework and RESTful API')
            with lm:
                lm.print_item('Flask')
                lm.print_item('Fast API')
                lm.print_item('Django')
        with lm:
            lm.print_item('Data Science')
            with lm:
                lm.print_item('NumPy')
                lm.print_item('Pandas or Polars')
                lm.print_item('Plotly')
                lm.print_item('Matplotlib')
                lm.print_item('Streamlit')
        with lm:
            lm.print_item('Machine Learning')
            with lm:
                lm.print_item('Sci-kit Learn')
                lm.print_item('TensorFlow or PyTorch')
                
        # Javascript
        lm.print_item('Javascript')
        with lm:
            lm.print_item('Javascript Fundamentals and Advance')
            with lm:
                lm.print_item('Closures, Scopes, and Functions')
                lm.print_item('Objects')
                lm.print_item('Higher Order Functions')
                lm.print_item('Data Structures')
                lm.print_item('Object-Oriented Programming')
                lm.print_item('Asynchronous Javascript')
            lm.print_item('Front-End Framework')
            with lm:
                lm.print_item('Typescript')
                lm.print_item('React JS')
                lm.print_item('React Native')
                lm.print_item('React Redux')
                lm.print_item('Vue')
                lm.print_item('Svelte')
            lm.print_item('Back-End Framework')
            with lm:
                lm.print_item('Node JS')
                lm.print_item('Express JS')

        # Databases 
        lm.print_item('Databases')
        with lm:
            lm.print_item('Relational Databases')
            with lm:
                lm.print_item('SQLite')
                lm.print_item('MySQL')
                lm.print_item('PostgreSQL')
            lm.print_item('Document and Object Databases')
            with lm:
                lm.print_item('GraphQL')
                lm.print_item('MongoDB')
                
        # Rust
        lm.print_item('Rust')
        with lm:
            lm.print_item('Rust Fundamentals')
            with lm:
                lm.print_item('Ownership and Lifetimes')
                lm.print_item('Structs and Enums')
                lm.print_item('Trait Implementations')
            lm.print_item('Web Assembly Frameworks')
            with lm:
                lm.print_item('Yew')
                lm.print_item('Tauri')
                lm.print_item('Rocket')
                
        # Deployment
        lm.print_item('Cloud and Deployment')
        with lm:
            lm.print_item('Github, Gitlab')
            with lm:
                lm.print_item('Git')
            lm.print_item('Docker')
            lm.print_item('Amazon Web Service')